In [1]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import Client, SSHCluster
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

# Macro-Pipeline AHN1 Workflow - Normalization


Choose whether you want to run all input files or run the only input files listed in `filename`.

In [2]:
path_root = pathlib.Path('/data/local/home/eecolidar_webdav/02_UvA/')

# path to retiled files 
path_input = path_root / 'YShi/AHN1/Retiled'

# path to normalized files
path_output = path_input.parent / 'normalized'

run = 'from_file' # 'all', 'from_file'
filename = 'normalize_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'from_file']

In [3]:
tiles = [el for el in path_input.iterdir() if el.match('tile_*_*/')]
print('Found: {} tiles'.format(len(tiles)))
if run == 'from_file':
    with open(filename, 'r') as f:
        tiles_read = json.load(f)
    tiles_read = [path_input/f for f in tiles_read]
    # check whether all files are available on dCache
    assert all([f in tiles for f in tiles_read]), f'Some of the tiles in {filename} are not in input dir'
    tiles = tiles_read
print('Normalize: {} tiles'.format(len(tiles)))

Found: 37715 tiles
Normalize: 8989 tiles


## Setup Cluster

Setup Dask cluster used for the macro-pipeline calculation.

In [4]:
local_tmp = pathlib.Path('/data/local/tmp')

nprocs_per_node = 2  

# start the cluster
scheduler_node = 'node1'
hosts = [f'node{i}' for i in range(1, 11)]
cluster = SSHCluster(hosts=[scheduler_node] + hosts, 
                     connect_options={'known_hosts': None, 
                                      'username': 'ubuntu', 
                                      'client_keys': '/home/ubuntu/.ssh/id_rsa'},
                     worker_options={'nthreads': 1, 
                                     'nprocs': nprocs_per_node,
                                     'memory_limit': 0,
                                     'local_directory': local_tmp/'dask-worker-space'}, 
                     scheduler_options={'dashboard_address': '8787'})
cluster

distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
distributed.deploy.ssh - INFO - warnings.warn(msg)
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at: tcp://145.100.59.123:8786
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.59:45041'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.59:41601'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         S

SSHCluster('tcp://145.100.59.123:8786', workers=18, threads=18)

## Normalization

Normalize the point heights.

In [5]:
# setup input dictionary to configure the normalization pipeline
normalization_input = {
    'setup_local_fs': {'input_folder': path_input.as_posix(),
                       'output_folder': path_output.as_posix()},
    'load': {'attributes': 'all'},
    # Filter out artifically high points - give overflow error when writing 
    'apply_filter': {'filter_type':'select_below',
                     'attribute': 'z',
                     'threshold': 10000.},  # remove non-physically heigh points
    'normalize': 1,
    'clear_cache' : {},
}

# write input dictionary to JSON file
with open('normalize.json', 'w') as f:
    json.dump(normalization_input, f)

In [6]:
macro = MacroPipeline()

# add pipeline list to macro-pipeline object and set the corresponding labels
for tile in tiles:
    dp = DataProcessing(tile.name, label=tile.name)
    normalization_input_ = copy.deepcopy(normalization_input)
    normalization_input_['export_point_cloud'] = {'filename': '{}.laz'.format(tile.name),
                                                  'overwrite': True}
    dp.config(normalization_input_)
    macro.add_task(dp)

macro.setup_cluster(cluster=cluster)

# run!
macro.run()

# save outcome results and check that no error occurred before continuing
macro.print_outcome(to_file='normalize.out')

failed = macro.get_failed_pipelines()
if failed:
    with open('normalize_failed.json', 'w') as f:
        json.dump([pip.label for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

## Terminate cluster

In [ ]:
macro.shutdown()

## Troubleshooting 

### Cancel all jobs and restart the notebook

Copy and paste these lines in a separate Python shell. If the Dask dashboard shows that some tasks are still queued to be processed, run the lines again - this should clear the scheduler up and give back control to the current notebook. Normally proceed to terminate the cluster and restart the notebook.

In [ ]:
from dask.distributed import Client, Future
client = Client('tcp://145.100.59.123:8786')
futures = [Future(key) for key in client.who_has().keys()]
client.cancel(futures)